In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, explode, when
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import time

if __name__ == "__main__":
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

    # read the tweet data from socket
    tweet_daf = spark \
        .readStream \
        .format("socket") \
        .option("host", "0.0.0.0") \
        .option("port", 5555) \
        .load()

    # type cast the column value
    tweet_df = tweet_daf.select(explode(split(col("value"), "TWEET__AN,")).alias("value"))
    tweet_df_string = (tweet_df.withColumn('tweet', split(tweet_df['value'], '____SEP,').getItem(0)).withColumn('location', split(tweet_df['value'], '____SEP,').getItem(1)).withColumn('favorites', split(tweet_df['value'], '____SEP,').getItem(2)).withColumn('followers', split(tweet_df['value'], '____SEP,').getItem(3)).withColumn('timestamp', split(tweet_df['value'], '____SEP,').getItem(4)).withColumn('name', split(tweet_df['value'], '____SEP,').getItem(5)))
    
    tweet_df_string =tweet_df_string.drop("value")
    tweet_df_sentiment = pipelineFit.transform(tweet_df_string).drop("tokens","filtered_words","vector","rawPrediction","probability")
    tweet_df_sentiment=tweet_df_sentiment.withColumnRenamed("prediction","sentiment")
    tweet_df_fl = pipelineFitBBC.transform(tweet_df_sentiment).drop("tokens","filtered_words","vector","rawPrediction","probability","cf")
    tweet_df_fl = tweet_df_fl.withColumn("favorites",col("favorites").cast("int")+1)
    tweet_df_fl = tweet_df_fl.withColumn("followers",col("followers").cast("int"))
    # write the above data into memory. consider the entire analysis in all iteration (output mode = complete). and let the trigger runs in every 2 secs.
    writeTweet = tweet_df_fl.writeStream.format("memory").outputMode("append").queryName("processed").trigger(processingTime ='30 seconds').start()
    print("----- streaming is running -------")


22/05/02 11:59:59 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


----- streaming is running -------


22/05/02 12:00:01 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/87/_c2ttbjn7tl1w8dzqfn6nz7w0000gn/T/temporary-6b907440-72f1-436e-8527-f007e9243295. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [15]:
spark.stop()

In [10]:
tweets = spark.sql("select * from processed where tweet!=''")

In [11]:
tweets.count()

992

In [ ]:
df=tweets.toPandas()
df.to_csv("output.csv")